In [1]:
import tkinter as tk
from tkinter import messagebox,filedialog
import unicodedata

import pyperclip
import numpy as np

#imports from the custom chord predictor libarary: 
from jl_dictionaries import Dictionaries
from jl_encoding import Encoder

Dictionaries.get_all_possible_chords()
vocab_size = len(Dictionaries.get_all_possible_chords())
vocab_size
encoder = Encoder(Dictionaries.get_all_possible_chords())


from keras.models import load_model
import tensorflow as tf
import numpy as np

Category to number dictionary length: 804
Number to category dictionary length: 804


In [2]:

model = load_model('lstm_normalised__W_20_lr_0_0005_epochs=50_batch_128.h5')
#model = load_model('lstm_normalised__W_20_lr_0_1_epochs=50_batch_128.h5')


In [3]:
model.output_shape

(None, 805)

In [4]:
def init_chords_options(type):
    global chords_options_list
    
    common_simple_chords=['A','B','C','D','E','F','G']
    common_simple_chords=[[c+'',c+'m',c+'#',c+'b'] for c in common_simple_chords]
    common_simple_chords =['delete']+[chord for chord_types in common_simple_chords for chord in chord_types if chord in encoder.number_to_category_dict.values()]    
    if type=='restricted':   
        #define what chords we wish to work with: (there are 804 options, but we only work with basic ones)
        chords_options_list = common_simple_chords
        #temp - remove chords that are not in the chords list: 
        print(chords_options_list)
    elif type=='all':
        chords_options_list=list(encoder.number_to_category_dict.values())
        #put the more common chords first in the list:
        # first, remove them from the entire list:  
        for chord in common_simple_chords[1:]: 
            chords_options_list.remove(chord)
        #then concatinate them (the common ones are first now) 
        chords_options_list=common_simple_chords+chords_options_list    
    else: 
        Exception('type method is non valid (must be either "all" or "restricted"')
    return chords_options_list

In [5]:
def pad_sequence(input, length):
    return [0] * (length - len(input)) + input

def get_predictions(model,prev_chords,allowed_chords,top_n=10):
    sequence=[encoder.to_number(chord) for chord in prev_chords]
    if len(sequence)>20:  #if we allready have more than 20 chords, just take the most recent 20 chords
        padded_sequence=sequence[-20:]
    else: 
        padded_sequence=pad_sequence(sequence,20)

    predicted_p_unsorted=model(np.array([padded_sequence]))
    predicted_p_unsorted_array=np.array(predicted_p_unsorted)
    descending_sorting_inds=np.argsort(-1*predicted_p_unsorted).flatten() #negative so sorting will result in descending order
    sorted_predicted_ps=(np.sort(-1*predicted_p_unsorted)*-1).flatten()

    allowed_chords_in_order=[ind for ind in descending_sorting_inds if (not(ind==0) and (encoder.number_to_category_dict[ind] in allowed_chords))]
    allowed_chords_proba_in_order=[predicted_p_unsorted_array[0][ind] for ind in descending_sorting_inds if (not(ind==0) and (encoder.number_to_category_dict[ind] in allowed_chords))]

    top_n_chords=[encoder.number_to_category_dict[chord] for chord in allowed_chords_in_order[0:top_n]]
    top_n_chords_p=allowed_chords_proba_in_order[0:top_n]
    return top_n_chords,top_n_chords_p

top_n_chords,top_n_chords_p=get_predictions(model=model,prev_chords=['C','F','F#','A','G','A','G']*6,allowed_chords=['G','F#','C'],top_n=2)
print('predicted chords:',top_n_chords)
print('predicted probabilities:',top_n_chords_p)

predicted chords: ['C', 'G']
predicted probabilities: [0.16737138, 0.040277347]


In [8]:
added_chords=[]
empty_chord_space_symbol='_'
max_words_in_sentence=8
how_many_cords_before_resorting_list=1
hebrew_flag=False
chords_options_list=init_chords_options('all')



def is_hebrew(term):
    return 'HEBREW' in unicodedata.name(term.strip()[0])

def on_double_click(event):
    global chords_options_list,added_chords
    how_many_cords_before_resorting_list=5
    chord = chords_list.get(chords_list.curselection())

    if not(chord=='delete'):
        added_chords.append(chord)
    print('selected chord:',chord)
    print('current state of chords list added by user',added_chords)

    chord_length = len(added_chords[-1])
    print('last chord length in list:',chord_length)
    #print(f'selected chord {chord} length {chord_length}')
    curr_text_string=output_text.get('1.0',tk.END)


    #get the chord left to the cursor by looking at the text string: 
    line_num, col_num = output_text.index('insert').split('.')
    print('selection location on the grid',(line_num, col_num))
    text_string_lines=curr_text_string.split('\n')
    line_num=int(line_num)-1
    col_num=int((int(col_num)-1))
    print('location_on_string:',(line_num,col_num))
    text_string_lines[line_num][col_num]
    print('selected text line string',text_string_lines[line_num])
    for m in np.arange(col_num,-1,-1):
        if text_string_lines[line_num][m]==empty_chord_space_symbol:
            break
    chord_left_to_selection_name_from_string=text_string_lines[line_num][m+1:col_num+1]
    print('chord left of cursor: ',chord_left_to_selection_name_from_string)


    #delete the previous chord from the same row it was inserted at
    if chord=='delete': #need to fix this, as i need to figure out how many characters the deleted code was, and to replace them with markers.
        chord_length=len(chord_left_to_selection_name_from_string)
        output_text.delete("insert -{}c".format(chord_length), "insert")
        print('state of chord list before deleting: ',added_chords) 
        print('last chord length=',chord_length)
        output_text.insert(tk.INSERT, empty_chord_space_symbol*chord_length)
        added_chords.remove(chord_left_to_selection_name_from_string) #need to see if i can find and delete it from the correct entree from the list and not just the last one
        print('state of chord list before deleting: ',added_chords)
    else:
        if len(chord_left_to_selection_name_from_string)>0: #altough the user didnt ask for delete operation, the cursor is placed right next to another chord so we will replace it
            
            chord_length=len(chord_left_to_selection_name_from_string)
            print(f'replacing {chord_left_to_selection_name_from_string} with {chord}')
            chord=empty_chord_space_symbol*(len(chord_left_to_selection_name_from_string)-len(chord))+chord
            print('after adding markers:',chord)
            chord_length=max(chord_length,len(chord))
            added_chords.remove(chord_left_to_selection_name_from_string)
        output_text.delete("insert -{}c".format(chord_length), "insert")
        output_text.insert(tk.INSERT, chord)

    
    print('reordering suggested chords list')
    top_n_chords,top_n_chords_p=get_predictions(model=model,prev_chords=added_chords,allowed_chords=chords_options_list,top_n=15)
    print('predicted chords',top_n_chords)
    chords_list1.delete(0,len(chords_options_list))
    chords_options_list1=top_n_chords
    for chord,chord_p in zip(chords_options_list1,top_n_chords_p):
        insertion_chord_and_probability=f'{chord} - {chord_p:.3f}%'
        chords_list1.insert(tk.END, insertion_chord_and_probability)
            
        

def on_paste_and_parse():
    text = pyperclip.paste()
    parsed_text = ""
    lines = text.split("\n")
    if len(lines)==1:
        words = text.split(" ")
        parsed_w_text = ""
        for w_cnt,word in enumerate(words):
            parsed_w_text = parsed_w_text +' '+ word
            if np.mod(w_cnt+1,max_words_in_sentence)==0:
                parsed_w_text=parsed_w_text+'\n' 
        lines = parsed_w_text.split("\n")

    largest_string=max(lines,key=len)
    #print('largest string:',largest_string)
    largest_string=len(largest_string)
    words_on_longest_line=lines[lines.index(max(lines,key=len))].split(' ')
    max_num_words_in_sentence=len(words_on_longest_line)
    #print('max words in longest setence:',type(max_num_words_in_sentence))
    
    #check if the language is hebrew: 
    hebrew_flag=any([is_hebrew(word) for word in words_on_longest_line if len(word)>1])
    if hebrew_flag: 
        print('detected hebrew')
    else: 
        print('hebrew not detected')
    
    empty_places_for_chords=largest_string+max_num_words_in_sentence
    parsed_text=empty_places_for_chords*empty_chord_space_symbol + '\n'
    for line in lines:
        if len(line)==0: #empty char, add a whole empty string symbols:
            line=empty_places_for_chords*empty_chord_space_symbol + '\n'
        if hebrew_flag:
            parsed_text += empty_places_for_chords*empty_chord_space_symbol + '\n' + line + "\n"
        else:
            parsed_text += line + "\n" + empty_places_for_chords*empty_chord_space_symbol + '\n'

    print(parsed_text)
    output_text.delete("1.0", tk.END)
    output_text.insert(tk.END, parsed_text)


def save_text():
    edited_text = output_text.get("1.0", "end") # Get the text from the text entry widget
    if not empty_chord_space_symbol==' ':
        edited_text=edited_text.replace(empty_chord_space_symbol,' ')
    print(edited_text)
    
    # Use a dialog box to get the file name to save to
    file_name = filedialog.asksaveasfilename(defaultextension=".txt",initialdir = ".\chord sheets outputs",filetypes=[("Text Files", "*.txt"), ("All Files", "*.*")])

    if file_name:
        with open(file_name, "w") as f:
            f.write(edited_text) # Write the text to a file
        print("Text saved successfully.")
    else:
        print("Save operation cancelled.")


root = tk.Tk()
root.title("Guitar Chords")
width  = int(root.winfo_screenwidth()/2)
height = int(root.winfo_screenheight()*0.85)
root.geometry(f'{width}x{height}+10+{int(height*0.025)}')


#######one chord list########

chords_frame = tk.Frame(root)
chords_frame.pack(side="left", fill="both", expand=True)

scrollbar = tk.Scrollbar(chords_frame, orient="vertical")
scrollbar.pack(side="left", fill="y")

chords_label=tk.Label(chords_frame,text='chords\nlist',height=2, width=8 , fg='black')
chords_label.configure(font=("TkFixedFont", 12, "bold"))
#sugested_chords=tk.Label(chords_frame,text='suggested\nchords',height=2, width=9)
#sugested_chords.configure(font=("TkFixedFont", 12, "bold"))
chords_list = tk.Listbox(chords_frame, xscrollcommand=scrollbar.set, height=1, width=10,bg='lavender',cursor="plus",highlightcolor='red',selectmode='SINGLE')

for chord in chords_options_list:
    chords_list.insert(tk.END, chord)
    
chords_label.pack(side='top',anchor='nw')
chords_list.pack(side="left", fill="both", expand=True)

#sugested_chords.pack(side='top',anchor='nw')
scrollbar.config(command=chords_list.yview)

######second chord list#########
chords_frame1 = tk.Frame(root)
chords_frame1.pack(side="left", fill="both", expand=True)
scrollbar1 = tk.Scrollbar(chords_frame1, orient="vertical")
#scrollbar1.pack(side="left", fill="y")

chords_label1 = tk.Label(chords_frame1,text='suggested\nchords',height=2, width=8 , fg='black')
chords_label1.configure(font=("TkFixedFont", 12, "bold"))
chords_list1 = tk.Listbox(chords_frame1, xscrollcommand=scrollbar1.set, height=1, width=15,bg='lavender blush',cursor="arrow")
for chord in []:
    chords_list1.insert(tk.END, chord)

chords_label1.pack(side='top',anchor='nw')
chords_list1.pack(side="left", fill="both", expand=True)
scrollbar1.config(command=chords_list1.yview)
chords_list_num_selected=np.zeros([len(chords_options_list)])
chords_list.bind("<Double-Button-1>", on_double_click)

output_frame = tk.Frame(root)
output_frame.pack(side="bottom", fill="both", expand=True)

output_text = tk.Text(output_frame, font=("TkFixedFont", 14),cursor="arrow",undo=True,autoseparators=True,maxundo=-1)
output_text.pack(side="top", fill="both", expand=True)
label = tk.Label(root,text = "Lyrics")
label.pack(side="top")

paste_and_parse_button = tk.Button(output_frame, text="Paste and Parse", command=on_paste_and_parse)
paste_and_parse_button.pack(side="bottom",fill="both", expand=False)

save_button = tk.Button(output_frame, text="Save", command=save_text)
save_button.pack(side="bottom", fill="both", expand=False)

root.mainloop()


detected hebrew
________________________________________________
________________________________________________
האלוהים הרגיש לפתע מן חולשה קלה:
________________________________________________
אז הרופא אמר לו - שמע זה בא מבטלה
________________________________________________
כי אלוהים פשוט היה נורא משועמם
________________________________________________
אז הוא ברא את העולם,
________________________________________________
ובגלל זה סובלים כולם.
________________________________________________

________________________________________________
בגן העדן חי אדם לבד ללא שינה
________________________________________________
וכשזכה סוף סוף, לישון בפעם ראשונה
________________________________________________
הוא לא ידע - כי זו היא מנוחתו האחרונה.
________________________________________________
כי כאשר בבוקר קם -
________________________________________________
ראה שיש לו גם...מאדאם!
________________________________________________

________________________________________________
אדם נשוי כל